# Agent Scratch Book

In [1]:
import logging
import os
import torch

from aicraft_agents.models import GPT, Llama3, Codellama


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
device

device(type='mps')

## Initialise Models

In [2]:
# llama3 = Llama3()
codellama = Codellama(device=device)
gpt = GPT()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
SYSTEM_PROMPT = """
Instruction:
You are postgres database developer who is instructed to write SQL queries to fetch data from the database. Your SQL is required to be efficient and not contain any syntax errors. Here are the DDLs for the database:

Metadata and DDLs:

Metadata about the table "behaviour.interest":
Consumer interests represented in fantastic_insights_dataset table  Dataset consists of 30,000+ user reviews from ~1000 audience panel members  Audience panel members are located in the United States and represent all major U.S. regions and demographics. The most represented demographics in the dataset are 18-35 males and 18-35 females in Southern California.  Each dataset row is a positive review of a product/service/content from users on our platform.';
comment on column behaviour.interest.gender is 'User gender: Male | Female | Non-Binary';
comment on column behaviour.interest.age_range is 'User age range: 13-17 | 18-24 | 25-34 | 35-44 | 45-54 | 55-64 |65+

DDL:
create table behaviour.interest (
  gender text, -- User gender: Male | Female | Non-Binary
  age_range text, -- User age range: 13-17 | 18-24 | 25-34 | 35-44 | 45-54 | 55-64 |65+ 
  city_name text, -- User city
  state_name text, -- User state
  country_name text, -- User country
  user_count integer, -- Number of users that endorse the review, for multiple endorsers
  subject text, -- Product, service, or content endorsed
  description text, -- Description of product, service, or content
  link text, -- Link to product, service, or content
  image_link text, -- Link to image of product, service, or content
  tag_1 text, -- User provided category for product, service or content
  tag_2 text, -- User provided category for product, service or content
  tag_3 text, -- User provided category for product, service or content
  tag_4 text, -- User provided category for product, service or content
  tag_5 text, -- User provided category for product, service or content
  tag_6 text, -- User provided category for product, service or content
  tag_7 text, -- User provided category for product, service or content
  tag_8 text, -- User provided category for product, service or content
  tag_9 text, -- User provided category for product, service or content
  tag_10 text -- User provided category for product, service or content
);

Strictly respond back only with the a SQL and not contain any syntax errors. 
"""

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "Respond back with SQL only: which age group have provided the highest number of endorsements?"},
]

In [4]:
# %%time
# response = llama3.generate(messages, temperature=0.9)
# logger.info(f'\n{response["content"]}')

In [5]:
%%time
response = gpt.generate(messages, temperature=0.9)
logger.info(f'\n{response["content"]}')

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:
```sql
SELECT age_range, SUM(user_count) AS total_endorsements
FROM behaviour.interest
GROUP BY age_range
ORDER BY total_endorsements DESC
LIMIT 1;
```


CPU times: user 16.9 ms, sys: 180 ms, total: 197 ms
Wall time: 1.48 s


In [6]:
%%time
response = codellama.generate(messages, temperature=0.9)
logger.info(f'\n{response["content"]}')

INFO:__main__:
 SELECT age_range, COUNT(*) as num_endorsements
FROM behaviour.interest
GROUP BY age_range
ORDER BY num_endorsements DESC
LIMIT 1;


CPU times: user 3.72 s, sys: 24.9 s, total: 28.6 s
Wall time: 39 s
